In [ ]:
# for reorganizing (and multithreading) aspect refinement 

handler function for single eclipse 

function for single frame of eclipse 

function for "normal" frame or "slew" frame 

function to collate aspect solutions from all frames 

In [ ]:
import sys
sys.path.insert(0, '/home/bekah/gPhoton2')
from gPhoton.reference import get_legs, titular_legs, check_eclipse, eclipse_to_paths
from gPhoton.types import Pathlike, GalexBand
from typing import Any, Callable, Sequence, Mapping, Optional, Literal
from pyarrow import parquet 
import pandas as pd 
import matplotlib.pyplot as plt 
from astropy.io import fits
import matplotlib.pyplot as plt 

In [ ]:
root = '/home/bekah/gPhoton2'
eclipse = 815 
band = "NUV"

tab= refine_eclipse(eclipse, band, root)

In [ ]:
plt.scatter(tab['frame_type'],tab.index)

In [ ]:
tab

In [ ]:
### Main handlers 

def refine_eclipse(
        eclipse: int, 
        band: str, 
        aspect_root: str, 
        ext="gzip"): 
    """ main pipeline for processing an eclipse """
    
    # setup to run pipeline by getting relevant info & paths 
    metadata_paths = metadata_filepaths(root)
    eclipse_info = get_eclipse_info(eclipse, metadata_paths)
    
    # paths for photonlist, possible output images etc 
    # function is from gPhoton2 
    paths = get_base_file_paths(
                eclipse=eclipse,
                band=band,
                ext=ext,
                root=aspect_root+'/test_data',
                mode='direct',
                legs=eclipse_info['actual_legs'])
    
    # get list of frames for all legs into a single dataframe, 
    # with paths for each backplane 
    frame_list = get_frame_list(eclipse, metadata_paths)
    modified_frame_list = pd.DataFrame()
    if eclipse_info['actual_legs'] == 0: 
            modified_frame_list = get_backplane_filenames(
                                        eclipse_info,
                                        frame_list,
                                        paths[0],
                                        0)        
    for leg in range(eclipse_info['actual_legs']): 
            files = get_backplane_filenames(
                        eclipse_info,
                        frame_list,
                        paths[leg],
                        leg)
            modified_frame_list = pd.concat([modified_frame_list, files], axis=0)
    
    #for frame in range(len(modified_frame_list)): 
            #aspect, warnings = refine_frame(modified_frame_list.iloc[frame])
            # aspect is tuple of ra, dec, roll, time 
            3#TODO: do I want the new aspect to go in the old mod_frame_list or it's own file?
            #modified_frame_list.iloc
            
    return modified_frame_list


def metadata_filepaths(root: str): 
    """return paths to aspect, boresight, and metadata parquet files """
    metadata_paths = {}
    metadata_paths['og_aspect'] = root + '/gPhoton/aspect/aspect.parquet'
    metadata_paths['expanded_aspect'] = root + '/gPhoton/aspect/aspect2.parquet'
    # astrom aspect doesn't exist yet but might use it 
    metadata_paths['astrometry_aspect'] = root + '/gPhoton/aspect/astrom_aspect.parquet'
    metadata_paths['boresight'] = root + '/gPhoton/aspect/boresight.parquet'
    metadata_paths['metadata'] = root + '/gPhoton/aspect/metadata.parquet'
    return metadata_paths 
    

def get_eclipse_info(
        eclipse: int, 
        metadata_paths: dict): 
    """returns a dictionary of info about eclipse"""
    actual, nominal = titular_legs(eclipse)
    eclipse_info = {}
    eclipse_info['actual_legs'] = actual 
    eclipse_info['nominal_legs'] = nominal 
    metadata = parquet.read_table(metadata_paths['metadata'],
                              filters=[('eclipse','==',eclipse)]).to_pandas()
    eclipse_info['ra_max'] = metadata['ra_max']
    eclipse_info['ra_min'] = metadata['ra_min']
    eclipse_info['dec_max'] = metadata['dec_max']
    eclipse_info['dec_min'] = metadata['dec_min']
    eclipse_info['obstype'] = metadata['obstype']
    
    return eclipse_info 
    
    
#TODO: need to check if legs naming starts with 0 or 1  
def get_base_file_paths(
        eclipse: int,
        band: GalexBand = "NUV",
        depth: Optional[int] = None,
        compression: Literal["none", "gzip", "rice"] = "gzip",
        root: Pathlike = "data",
        start: Optional[float] = None,
        mode: str = "direct",
        legs: int = 0,
        aperture: Optional[float] = None,
        **kwargs,
    ) -> dict[str, str]:
    """dictionary of dictionaries of file paths for each leg of eclipse """
    
    paths = {} 
    if legs == 0: 
        paths[0] = eclipse_to_paths(eclipse=eclipse,band=band,depth=1,
                 compression=compression,root=root,mode=mode,leg=legs,) 
    else: 
        for i in range(legs): 
            paths[i] = eclipse_to_paths(eclipse=eclipse,band=band,depth=1,
                 compression=compression,root=root,mode=mode,leg=i) 
    return paths 


def get_frame_list(
        eclipse:int, 
        metadata_paths: dict): 
    """ join extended and og aspect parquet per eclipse to get unrefined 
    'slew' frames """
    from pyarrow import parquet 
    og_aspect = parquet.read_table(metadata_paths['og_aspect'],
                                  filters=[('eclipse','==',eclipse)]).to_pandas()
    og_aspect['original'] = 'ref'

    new_aspect = parquet.read_table(metadata_paths['expanded_aspect'],
                                  filters=[('eclipse','==',eclipse)]).to_pandas()
    new_aspect = new_aspect.rename(columns={"pktime": "time"})
    new_aspect['original'] = 'slew'

    new_aspect = new_aspect.merge(og_aspect, on="time",how="outer") 

    new_aspect['original_y'].fillna('slew',inplace=True)
    new_aspect = new_aspect.rename(columns={"original_y": "frame_type"})

    # get rid of weird distant time stamp at end of file sometimes
    # (only works if it's one entry)
    if new_aspect.iloc[-1].time - new_aspect.iloc[-2].time > 10: 
        new_aspect.drop(new_aspect.tail(1).index,inplace=True)
    print(new_aspect)
    return new_aspect


def get_backplane_filenames(
        eclipse_info: dict,
        frame_list, 
        paths: dict,
        leg: int): 
    """ produce df of backplane filenames """
    phot = parquet.read_table(
            paths['photonfile'], columns=['t', 'col', 'row', 'detrad']
        )
    t_f = phot['t'][0].as_py() # first timestamp of leg from photonlist
    t_l = phot['t'][-1].as_py() # last timestamp of leg from photonlist
    
    #TODO: trying to change this 
    #leg_frames = frame_list.copy()
    leg_frames = frame_list.loc[(frame_list['time'] >= t_f) & (frame_list['time'] <= t_l)].copy()
    
    # backplanes name formatting fNNNNdd_tNNNNdd
    # this is probably an overly complex way to do the naming but ... it works 
    leg_frames['backplane_path'] = paths['movie'].replace('.fits.gz', f'_dose.fits')\
        .replace('movie','tmovie')
    # subtract first timestamp of photonlist 
    leg_frames['time'] = leg_frames['time']-t_f
    leg_frames['time_stamp'] = leg_frames['time'].astype(str).str.split('.').str[0]\
        .str.zfill(4).str.cat(leg_frames['time'].astype(str).str.split('.').str[1].str[:4])
    leg_frames['backplane_path'] = leg_frames['backplane_path'].str.split('movie').str[0]\
        .str.cat(leg_frames['time_stamp']).str.cat(leg_frames['backplane_path']\
        .str.split('movie').str[1])
    leg_frames['leg'] = leg 
    # xylist path = l[leg]ts[time stamp].xyls
    leg_frames['xylist_path'] = "l"+leg_frames['leg'].astype(str)+"ts"+\
        (leg_frames['time_stamp'].astype(str))+".xyls"
    return leg_frames 


def leg_start_cleanup(): 
    """ ok so the first time stamp in the photonlist doesn't necessarily perfectly 
    match a time stamp in the aspect table. why is that? Idk, it's annoying, might 
    have to do with some kind of rounding happening somewhere in python. """

In [ ]:
from astropy.io import fits 
img = fits.open("/media/bekah/Extreme Pro/e00815/e00815-nd-b00-f0001-t00180-g_dose.fits.gz")

In [ ]:
img[0].header['NAXIS1']

In [ ]:
m = range(120,150)
slopes = [(tab.iloc[t+1]['ra_acs']-tab.iloc[t]['ra_acs'])/(tab.iloc[t+1]['dec_acs']-tab.iloc[t]['dec_acs']) for t in m]

In [ ]:
plt.scatter(tab.iloc[120:150]['roll'],slopes)

In [ ]:
# SCST download 
""" Download SCST file from MAST """
sys.path.insert(0, '/home/bekah/gPhoton2')
from gPhoton.io.mast import get_raw_paths 
# scst files 
paths = get_raw_paths(10982)
paths['scst']

In [ ]:
scst = fits.open('/home/bekah/Downloads/e10982-scst (1).fits.gz')

In [ ]:
scst[1].header

In [ ]:
tab['time_diff'] = tab['time'].diff()
tab['time_diff'].describe()

In [ ]:
def stack_frames(streaks, frame_series, slew_paths, direction):
    """looks at ASTRIDE results and uses streak length / direction to
    stack .1s frames into a 1s image """
    layers = 10 # how many .1 s frames we add together, don't have to hard code here
    hdul = fits.open(frame_series['backplane_path'])
    img1 = hdul[0].data
    new_shape = ((layers - 1) * streaks['y_offset'] + img1.shape[0],
                 (layers - 1) * streaks['x_offset'] + img1.shape[1])

    stacked = np.zeros(new_shape)  # , dtype=np.float)
    stacked2 = np.zeros(new_shape)  # , dtype=np.float)
    # adding image layers together
    #TODO: edit for correct backplane names in loop for short ones
    # ADD DIRECTION
    for layer in range(layers):
        print("adding image layers together")
        dose_ais = fits.open(slew_paths['short_backplanes'][layer])
        img1 = dose_ais[0].data  # *layer # for tagging pixels as being from a layer
        layer_op = (layers - 1) - layer
        stacked[layer_op * streaks['y_offset']:layer_op * streaks['y_offset'] + img1.shape[0],
        layer_op * streaks['x_offset']:layer_op * streaks['x_offset'] + img1.shape[1]] += img1
    # saving image to fits
    hdu = fits.PrimaryHDU(stacked)
    hdul = fits.HDUList([hdu])
    hdul.writeto(slew_paths["stacked"], overwrite=True)
    
     for layer in range(layers):
        print("adding image layers together")
        dose_ais = fits.open(slew_paths['short_backplanes'][layer])
        img1 = dose_ais[0].data  # *layer # for tagging pixels as being from a layer
        layer_op = (layers - 1) - layer
        stacked[layer_op * streaks['y_offset']:layer_op * streaks['y_offset'] + img1.shape[0],
        layer_op * streaks['x_offset']:layer_op * streaks['x_offset'] + img1.shape[1]] += img1
    # saving image to fits
    hdu = fits.PrimaryHDU(stacked)
    hdul = fits.HDUList([hdu])
    hdul.writeto(slew_paths["stacked"], overwrite=True)
    
    return


In [ ]:
def stack_frames(streaks, frame_series, slew_paths, direction):
    # num frames 
    num_frames = 10 

    # new image size is the offset between frames * the number of frames being added 
    # plus the og image size 
    # num # frames is 10 usually (.1s increments for 1s)
    x_dim = ((num_frames - 1)* x_offset + first_frame.shape[0]) 
    y_dim = ((num_frames - 1)* y_offset + first_frame.shape[1]) 

    # empty image of stacked dimensions 
    stacked = np.zeros((x_dim, y_dim))  
    stacked_opposite = np.zeros((x_dim, y_dim))  

    # iterate through adding 10 .1 s frames 
    for frame in range(num_frames):
        # get frame image 
        frame_hdul = fits.open(slew_paths['short_backplanes'][frame])
        frame_image = frame_hdul[0].data
        # countdown layers 
        inverse_layer = (num_frames - 1) - frame
        # add frame to section of stacked image that corresponds with offset 
        stacked[inverse_layer * streaks['y_offset']:inverse_layer * streaks['y_offset'] + \
                frame_image.shape[0],
                inverse_layer * streaks['x_offset']:inverse_layer * streaks['x_offset'] + \
                frame_image.shape[1]] += frame_image
        # add frame to section of stacked image that corresponds with offset in opposite dir
        stacked_opposite[frame * streaks['y_offset']:frame * streaks['y_offset'] + \
                frame_image.shape[0],
                frame * streaks['x_offset']:frame * streaks['x_offset'] + \
                frame_image.shape[1]] += frame_image

    # saving image to fits
    hdu = fits.PrimaryHDU(stacked)
    hdul = fits.HDUList([hdu])
    hdul.writeto(slew_paths["stacked"], overwrite=True)
    hdu = fits.PrimaryHDU(stacked_opposite)
    hdul = fits.HDUList([hdu])
    hdul.writeto(slew_paths["stacked_opposite"], overwrite=True)  

    return 

In [ ]:
layers = 10 
for layer in range(layers):
    layer_op = (layers - 1) - layer
    print(layer_op)

In [ ]:
new_as = pd.read_csv('new_aspect.csv')
back_t = pd.read_csv('backplane_times.csv')


In [ ]:
new_as

In [ ]:
back_t

In [ ]:
photonfile = "/media/bekah/BekahA/glcat/e00815/e00815-nd-b05.parquet"

phot = parquet.read_table(
            photonfile, columns=['t', 'col', 'row', 'detrad']
        )
t_f = phot['t'][0].as_py() # first timestamp of leg from photonlist
t_l = phot['t'][-1].as_py() # last timestamp of leg from photonlist

In [ ]:
t_f = round(t_f,3)

In [ ]:
t_f

In [ ]:
t_f = round(t_f,3)
leg_frames = new_as.loc[(new_as['time'] >= t_f) & (new_as['time'] <= t_l)].copy()
leg_frames = leg_frames.round({'time': 3})


In [ ]:
leg_frames.iloc[0]['time']

In [ ]:
leg_frames.iloc[191]['time']

In [ ]:
back_t = back_t.round({'time': 3})
back_t['time'].iloc[0]

In [ ]:
back_t['time'].iloc[1]-leg_frames.iloc[1]['time']

In [ ]:
new_as['time'].iloc[1305]

In [ ]:
back_t.iloc[192]['time']

In [ ]:
new = leg_frames.merge(back_t, on="time", how="outer")
new

In [ ]:
streaks = pd.read_csv("/media/bekah/BekahA/glcat/e00815/astrom/streaks.txt", delim_whitespace=True)


In [ ]:
streaks

In [ ]:
def truncate(num, places):
    mult = 10 ** places
    return int(num * mult) / mult

In [ ]:
truncate(740388266.99504,3)

In [ ]:
truncate(740388511.99998,3)

In [ ]:
truncate(740389281.995,3)

In [ ]:
truncate(740389281.9,3)

In [ ]:
xylist = fits.open('/media/bekah/BekahA/glcat/e01532/astrom/l0ts01130.xyls')
xylist

In [ ]:
xylist[1].data

In [ ]:
from aspect_correction.util import get_aspect_from_wcsinfo 

In [ ]:
get_aspect_from_wcsinfo('/media/bekah/BekahA/glcat/e00815/astrom/l0ts00040.wcs')

In [ ]:
hdu = fits.open('/media/bekah/BekahA/glcat/e01532/astrom/l0ts00360.xyls')
hdu[1].data

In [ ]:
hdu[0].header['COMMENT'].data

In [ ]:
asp815 = pd.read_csv("/media/bekah/BekahA/glcat/e00815/00815_new_aspect.csv")

In [ ]:
asp815

In [8]:
def astrometry_hostess_run(): 
    from hostess.subutils import Viewer
    solve_process = Viewer.from_command(
        "solve-field",
        "/media/bekah/BekahA/glcat/e00815/astrom/l0ts00040.xyls",
        overwrite=True,
        no_plots=True,
        dir_="/media/bekah/BekahA/glcat/e00815/astrom/",
        width=1200,
        height=1200,
        scale_units="arcsecperpix",
        scale_low=1.0,
        scale_high=1.6,
        N="none",
        U="none",
        B="none",
        M="none",
        R= "none",
        _3=120,
        _4=120,
        temp_axy=True,
        crpix_x=600,
        crpix_y=600)
    solve_process.wait()
    return solve_process.done
fun()

True

In [13]:
solve_process.runner

In [15]:
from hostess.utilities import timeout_factory
waiting, _ = timeout_factory(timeout=100)

In [17]:
waiting()

0

In [18]:
waiting()

6.621674299240112

In [19]:
waiting()

11.627882480621338

In [20]:
waiting()

20.03666853904724

In [43]:
def astrometry_hostess_run(): 
    from hostess.subutils import Viewer
    solve_process = Viewer.from_command(
        "solve-field",
        "/media/bekah/BekahA/glcat/e00815/astrom/l0ts00040.xyls",
        overwrite=True,
        no_plots=True,
        dir_="/media/bekah/BekahA/glcat/e00815/astrom/",
        width=1200,
        height=1200,
        scale_units="arcsecperpix",
        scale_low=1.0,
        scale_high=1.6,
        N="none",
        U="none",
        B="none",
        M="none",
        R= "none",
        _3=120,
        _4=120,
        temp_axy=True,
        crpix_x=600,
        crpix_y=600)
    solve_process.wait()
    return solve_process.done


def time_run(): 
    from hostess.profilers import DEFAULT_PROFILER as PRO 
    with PRO.context("test_1"): 
        astrometry_hostess_run()
    return PRO.labels['test_1']['time']

In [44]:
time_run()

0.151